In [ ]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# Download necessary data for tokenization
nltk.download('punkt_tab')
nltk.download('stopwords')

stop_words = set(stopwords.words('french'))
stop_words.update(['@', '#', '€','!','rt',':','a'])
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

df_g = pd.read_csv("/content/drive/MyDrive/projet/Tweet2.csv", on_bad_lines='skip', sep=';')
df_g = df_g[ 'data__text']
df_g = df_g.dropna()

df_d = pd.read_csv("/content/drive/MyDrive/projet/Tweet1.csv", on_bad_lines='skip',sep=';')
df_d = df_d[ 'data__text']
df_d = df_d.dropna()

G_train, G_test  = train_test_split(df_g, test_size=0.99, random_state=42)
D_train, D_test = train_test_split(df_d, test_size=0.99, random_state=42)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:

# Optimisation de la fonction `remove_stopwords`
def remove_stopwords(text, stop_words):
    return [word.lower() for word in word_tokenize(text) if word.lower() not in stop_words]

# Optimisation de la fonction `remplir`
def remplir(counter, words):
    for word in words:
        counter[word] = counter.get(word, 0) + 1

#remplie les listes / dico pour la vectorisation
def remplir_dico(dataset,dico_g,dico_local,liste_trie):
    for sentence in dataset:
        new=remove_stopwords(sentence,stop_words)
        remplir(dico_local,new)
        remplir(dico_g,new)
        liste_trie.append(new)

# Préparation des dictionnaires
dictio = {}
dictio_g = {}
dictio_g_test = {}
dictio_d = {}
dictio_d_test = {}
add_g = []
add_g_test=[]
add_d = []
add_d_test=[]

# Traitement des phrases
remplir_dico(G_train,dictio,dictio_g,add_g)
remplir_dico(G_test,dictio,dictio_g_test,add_g_test)
remplir_dico(D_train,dictio,dictio_d,add_d)
remplir_dico(D_test,dictio,dictio_d_test,add_d_test)



In [ ]:
from scipy.sparse import csr_matrix

#fonction de passage dans l'espace
def faire_matrice (matrice,liste_texte,dico,index,start):
    for i,sentence in enumerate(liste_texte):
        for word in sentence:
            matrice[start+i,index[word]]=dico[word]

# Initialisation de la matrice
l = len(dictio)
l1 = len(G_train)  # Nombre de phrases dans G_train
l1_test = len(G_test)
print(l1_test)
l2 = len(D_train)  # Nombre de phrases dans D_train
l2_test=len(D_test)

# Construction de la matrice pour G_train
word_to_index = {word: i for i, word in enumerate(dictio)}  # Dictionnaire mot -> index
#print(word_to_index['j'])
#print(dictio['j'])
#print(word_to_index['j'])
matrice_A = np.zeros((l1 + l2, l), dtype=np.int16)
faire_matrice(matrice_A,add_g,dictio_g,word_to_index,0)
faire_matrice(matrice_A,add_d,dictio_d,word_to_index,l1)

matrice_A_prime = np.zeros((l1_test + l2_test, l), dtype=np.int16)
faire_matrice(matrice_A_prime,add_g_test,dictio_g_test,word_to_index,0)
faire_matrice(matrice_A_prime,add_d_test,dictio_d_test,word_to_index,l1_test)

sparse_matrice_A = csr_matrix(matrice_A)
sparse_matrice_A_prime = csr_matrix(matrice_A_prime)
print(sparse_matrice_A)
#print(sparse_matrice_A.shape)

5738
  (0, 0)	7
  (0, 1)	71
  (0, 2)	1
  (0, 3)	1
  (0, 4)	1
  (0, 5)	1
  (0, 6)	1
  (0, 7)	1
  (0, 8)	2
  (0, 9)	1
  (0, 10)	56
  (0, 11)	1
  (0, 12)	1
  (0, 13)	2
  (0, 14)	1
  (0, 15)	1
  (0, 16)	2
  (0, 17)	2
  (0, 18)	1
  (0, 19)	1
  (0, 20)	1
  (0, 21)	5
  (0, 22)	4
  (0, 23)	1
  (0, 24)	1
  :	:
  (172, 99)	1
  (172, 139)	3
  (172, 181)	21
  (172, 186)	21
  (172, 404)	2
  (172, 1008)	3
  (172, 1062)	2
  (172, 4887)	1
  (172, 6043)	1
  (172, 6776)	2
  (172, 7322)	1
  (172, 9383)	2
  (172, 10045)	1
  (172, 11212)	1
  (172, 11353)	1
  (172, 14134)	1
  (172, 15118)	1
  (172, 16779)	1
  (172, 21203)	1
  (172, 21204)	1
  (172, 21205)	1
  (172, 21206)	1
  (172, 21207)	1
  (172, 21208)	1
  (172, 21209)	1


In [ ]:
#Y = np.hstack((np.ones(l1), np.zeros(l2)))  # Labels (1: positif, 0: négatif)
#model = LogisticRegression()
#model.fit(matrice_A, y)
#coefficients = np.linalg.lstsq(matrice_A, Y, rcond=None)[0]

# Afficher les résultats
#print("Matrice des coefficients :\n", coefficients)



###############################################################
matrice_g=np.zeros(l)
matrice_d=np.zeros(l)

for i,clef in enumerate(dictio) :
    if clef in dictio_g:
        matrice_g[i]=dictio_g[clef]/dictio[clef]
    if clef in dictio_d:
        matrice_d[i]=dictio_d[clef]/dictio[clef]

l1_test = l1_test+0

In [ ]:
from scipy.sparse.linalg import lsqr
from sklearn.preprocessing import MaxAbsScaler

#matrice_A_normalized = scaler.fit_transform(sparse_matrice_A)
scaler = MaxAbsScaler()
matrice_A_normalized = scaler.fit_transform(sparse_matrice_A)
# Entraînement
Y = np.hstack((np.ones(l1), np.zeros(l2)))  # Labels (1: positif, 0: négatif)
coefficients = lsqr(matrice_A_normalized, Y)[0]

# Résultats des coefficients
print("Matrice des coefficients :\n", coefficients)
l1_test = l1_test+0

Matrice des coefficients :
 [-0.03886026  0.00389625  0.01863329 ...  0.          0.
  0.        ]


In [ ]:
l1_test = l1_test+0
import numpy as np
# Fonction pour arrondir
def arrondir(colonne):
    return (colonne >= 0.5).astype(int)

# Test

Y_test = np.hstack((np.ones(l1_test), np.zeros(l2_test)))  # Labels (1: positif, 0: négatif)
Y_pred = sparse_matrice_A_prime @ coefficients  # Utilise l'opérateur @
Y_pred=Y_pred.flatten()
#print("La colonne Y est :", Y_test)
#print("La colonne Y_pred est :", Y_pred)
Y_pred = arrondir(Y_pred)

# Validation
validation = Y_test - Y_pred
good = np.sum(validation == 0)
not_good = np.sum(validation != 0)

print("Nb de bons résultats :", good, "nb de mauvais :", not_good, "sur :", len(validation))
print("ratio =",good/len(validation))


Nb de bons résultats : 6400 nb de mauvais : 10869 sur : 17269
ratio = 0.3706062887254618


In [ ]:
import numpy as np
def arrondir(colonne):
    colonne_bis=np.zeros(len(colonne))
    for i in range (len(colonne)):
        if colonne[i]<0.5 :
            colonne_bis[i]=0
        else :
            colonne_bis[i]=1
    return(colonne_bis)

Y_test = np.hstack((np.ones(l1_test), np.zeros(l2_test)))

Y_pred=arrondir(matrice_A_prime*coefficients)
print("La colonne Y est :", Y_test)
print("La colonne Y_pred est :", Y_pred)
validation=Y_test-Y_pred

good,not_good=0,0
for element in validation:
    if element==0:
        good+=1
    elif element==1:
        not_good+=1

print("Nb de bon résultat :",good,"nb de mauvais :",not_good,"sur :",len(validation))

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()